# Extração da tabela da página da CVM

O objetivo é extrair do site da CVM a tabela com todos os TCs celebrados. Para isso, acabamos utilizando um driver e por meio da iteração entre as páginas, chegamos em uma tabela final.

-------------

Primeiro, vamos tentar extrair usando apenas pandas.

In [1]:
import pandas as pd

# URL da página com a tabela
url = "https://conteudo.cvm.gov.br/termos_compromisso/index.html"

# Extrair todas as tabelas da página
tabelas = pd.read_html(url)

# Caso a página contenha mais de uma tabela, selecionar a tabela correta
# (no exemplo abaixo, estamos assumindo que a tabela desejada é a primeira)
tabela = tabelas[0]

# Exibir as primeiras linhas da tabela extraída
print(tabela.head())

                                             0                  1  \
0                           Número do processo  Data de aprovação   
1                         19957.014281/2023-10         26/11/2024   
2                         19957.002548/2024-07         26/11/2024   
3  19957.000907/2024-83 e 19957.000887/2024-41         29/10/2024   
4                         19957.001633/2024-40         29/10/2024   

                    2           3                                     4  \
0  Data de assinatura  Publicação                        Compromitentes   
1          06/01/2025  13/01/2025               Flávio Vidigal de Capua   
2          06/01/2025  13/01/2025  Gustavo Henrique Paganoto Moscatelli   
3          16/12/2024  18/12/2024                Gustavo Duarte Pimenta   
4          16/12/2024  18/12/2024                Gustavo Duarte Pimenta   

                                       5                     6  
0  Decisão do Colegiado e Parecer do CTC  Data de arquivamento  
1   

Extraímos somente a primeira página dos TCs.

-------------------

Vamos ter e usar o webdriver e iterar pelas páginas.

In [15]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
import time
import pandas as pd
from selenium import webdriver

# Inicializa o WebDriver
driver = webdriver.Chrome()
url = "https://conteudo.cvm.gov.br/termos_compromisso/index.html"
driver.get(url)

# Lista para armazenar os dados
todas_as_tabelas = []

while True:
    # Aguarda a tabela carregar
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
    
    # Extrai a tabela da página atual
    html = driver.page_source
    tabelas = pd.read_html(html)
    todas_as_tabelas.append(tabelas[0])
    print("Tabela extraída com sucesso!")
    
    try:
        # Aguarda e busca o botão "Próxima" sempre que for clicar
        botao_proxima = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Próxima')]"))
        )
        
        botao_proxima.click()
        
        # Aguarda até que o botão anterior se torne obsoleto (stale)
        WebDriverWait(driver, 10).until(EC.staleness_of(botao_proxima))
        
        # Opcional: aguarda a nova tabela carregar
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
        
    except Exception as e:
        print("Erro ao clicar no botão 'Próxima':", e)
        break

# Fecha o navegador
driver.quit()

# Concatena todas as tabelas em um único DataFrame
tabela_final = pd.concat(todas_as_tabelas, ignore_index=True)
print(tabela_final)


Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraída com sucesso!
Tabela extraíd

In [16]:
tabela_final

,0,1,2,3,4,5,6
0,Número do processo,Data de aprovação,Data de assinatura,Publicação,Compromitentes,Decisão do Colegiado e Parecer do CTC,Data de arquivamento
1,19957.014281/2023-10,26/11/2024,06/01/2025,13/01/2025,Flávio Vidigal de Capua,Decisão/Parecer,27/01/2025
2,19957.002548/2024-07,26/11/2024,06/01/2025,13/01/2025,Gustavo Henrique Paganoto Moscatelli,Decisão/Parecer,27/01/2025
3,19957.000907/2024-83 e 19957.000887/2024-41,29/10/2024,16/12/2024,18/12/2024,Gustavo Duarte Pimenta,Decisão/Parecer,15/01/2025
4,19957.001633/2024-40,29/10/2024,16/12/2024,18/12/2024,Gustavo Duarte Pimenta,Decisão/Parecer,15/01/2025
...,...,...,...,...,...,...,...
984,Número do processo,Data de aprovação,Data de assinatura,Publicação,Compromitentes,Decisão do Colegiado e Parecer do CTC,Data de arquivamento
985,SP96/0072 - PRIMUS CORRETORA,11/12/1998,14/12/1998,17/02/1999,Primus Corretora e Hugo Barreto del Priore,NaN,NaN
986,SP97/00250 - NOVAÇÃO,03/12/1998,05/04/1999,20/07/1999,Júlio Manuel Villariço,NaN,NaN
987,SP97/00250 - NOVAÇÃO,03/12/1998,31/03/1999,10/05/1999,Novação S/A Corretora e Renata Rizkallah,NaN,NaN


In [17]:
# Exporta o DataFrame para um arquivo CSV
tabela_final.to_csv('processos_web_scraping.csv', index=False, encoding='utf-8')